In [1]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import pyodbc
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
import streamlit as st

In [2]:
pd.set_option('display.float_format', '{:.2f}'.format)

# TASK1 :Đọc dữ liệu

### 1.1 Đọc dữ liệu trong thư mục có bao nhiêu file

In [3]:
source_file = "C:/Users/lesyd/OneDrive/MWG_BHX/Theo dõi PX/PX/"

In [4]:
excel_file_list = os.listdir(source_file)

In [5]:
df = pd.DataFrame()

In [6]:
start_time = datetime.now()

for excel_files in excel_file_list:
    if excel_files.endswith(".xlsx"):
        strfile = source_file + excel_files
        conn_str = (r'Driver={{Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)}};'
            r'DBQ={}; ReadOnly=0').format(strfile)
        conn =  pyodbc.connect(conn_str, autocommit=True)
        crsr = conn.cursor()
    for worksheet in crsr.tables():
        str_sql = "SELECT [Mã siêu thị],[Tên siêu thị],[Ngày xuất],[Mã sản phẩm],[Tên sản phẩm],[Số lượng],[Đơn giá xuất],[retailprice],[Hình thức xuất] FROM "  + "[" + worksheet[2] + "]"
        df1 = pd.read_sql(str_sql, conn)
        df = df.append(df1)
    conn.close()
        
end_time = datetime.now()
print('Run time: {}'.format(end_time - start_time)) 

Run time: 0:02:09.832920


In [7]:
df['Số lượng'] = df['Số lượng'].astype(float).sum()

In [8]:
df['Ngày xuất'] = pd.to_datetime(df['Ngày xuất']).dt.date

In [9]:
df.loc[df["Đơn giá xuất"]<df["retailprice"],"Tình trạng"] ="Giảm giá "
df.loc[df["Đơn giá xuất"]==df["retailprice"],"Tình trạng"] ="Nguyên giá "

In [10]:
table = pd.pivot_table(df, values='Số lượng', index=['Ngày xuất'],
                    columns=['Tình trạng'], aggfunc=np.sum)

In [11]:
df2 = table.reset_index()

In [12]:
fig = px.bar(df2, x='Ngày xuất', y=['Giảm giá ', 'Nguyên giá '],barmode='group')
fig.show()

In [13]:
df['Hình thức xuất'].unique()

array(['Xuất bán hàng tại siêu thị', 'Xuất bán nội bộ'], dtype=object)

In [14]:
table_SL = pd.pivot_table(df, values='Số lượng', index=['Tên sản phẩm'],
                    columns=['Tình trạng'], aggfunc=np.sum).reset_index()

In [15]:
fig = px.bar(table_SL.sort_values(by='Nguyên giá ', ascending=False).head(10), y='Tên sản phẩm', x=['Giảm giá ', 'Nguyên giá '],barmode='group',orientation='h')
fig.show()

In [17]:
st.write(fig)

2022-08-26 22:10:20.834 
  command:

    streamlit run c:\Users\lesyd\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
